> TL;DR <br>
В коде находятся:
* Catboost для разных видов задач + optuna + feats. importance
* Xgboost для разных видов задач + optuna
* Stacking
* Voiting ensemble
* Всякая предобработка данных (Fill NaN, feats. selection, etc.)
* Другое (Кроссвалидация, подбор трешхолда, etc.)

In [7]:
# imports
import pandas as pd
import numpy as np
import catboost
import xgboost
from sklearn.preprocessing import PolynomialFeatures
from sklearn.experimental import enable_iterative_imputer
from sklearn.linear_model import LinearRegression
from sklearn.impute import IterativeImputer
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import SMOTE
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import VarianceThreshold
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_predict
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.metrics import f1_score, classification_report
from sklearn.metrics import roc_auc_score, precision_recall_curve
import optuna

In [4]:
# your df
X_train, y_train, X_val, y_val, X_test, y_test = 0, 0, 0, 0, 0, 0

# **Catboosts**

## Defult catboost

**Catboost params**:
* eval_metric {'F1', 'RMSE'}
* auto_class_weights {'default', 'Balanced', 'SqrtBalanced'}
* text_features {None, text_features}
* loss_function {'MultiClass', 'LogLoss', 'CrossEntropy'}

In [ ]:
# classification
cat_classif = catboost.CatBoostClassifier(eval_metric='F1', iterations=1000, random_seed=42, 
                                             task_type='GPU', auto_class_weights='default')

cat_classif.fit(X_train, y_train, eval_set=(X_val, y_val),
             verbose=100, early_stopping_rounds=300)

In [ ]:
# regressor
cat_reg = catboost.CatBoostRegressor(eval_metric='RMSE', iterations=1000, random_seed=42, 
                                             task_type='GPU', auto_class_weights='default')

cat_reg.fit(X_train, y_train, eval_set=(X_val, y_val),
             verbose=100, early_stopping_rounds=300)

## Catboost Optuna

In [ ]:
def objective(trial):
    param = {
        'iterations': trial.suggest_int('iterations', 500, 5000),
        "objective": trial.suggest_categorical("objective", ["Logloss"]),
        "colsample_bylevel": trial.suggest_float("colsample_bylevel", 0.01, 0.1),
        "learning_rate": trial.suggest_float("learning_rate", 1e-3, 1, log=True),
        "depth": trial.suggest_int("depth", 1, 12),
        "bootstrap_type": trial.suggest_categorical("bootstrap_type", ["Bayesian", "Bernoulli", "MVS"])
        }

    if param["bootstrap_type"] == "Bayesian":
        param["bagging_temperature"] = trial.suggest_float("bagging_temperature", 0, 20)
    elif param["bootstrap_type"] == "Bernoulli":
        param["subsample"] = trial.suggest_float("subsample", 0.1, 1)
    if param["objective"] == "Logloss":
        param["objective"] = trial.suggest_categorical("auto_class_weights", ["default", "Balanced", "SqrtBalanced"])
        
    cat_cls = catboost.CatBoostClassifier(**param, eval_metric='F1')

    cat_cls.fit(X_train, y_train, eval_set=[(X_val, y_val)] ,verbose=0, early_stopping_rounds=500)
    
    preds = cat_cls.predict(X_test)
    f1 = f1_score(y_test, preds)
    return f1

if __name__ == "__main__":
    study = optuna.create_study(direction="maximize")
    study.optimize(objective, timeout=7200)

    print("Number of finished trials: {}".format(len(study.trials)))

    print("Best trial:")
    trial = study.best_trial

    print("  Value: {}".format(trial.value))

    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))

In [ ]:
sorted_trials = sorted(study.trials, key=lambda trial: -trial.value)
top_10_trials = sorted_trials[:50] 
top_trials_params = []
for trial in top_10_trials:
    top_trials_params.append(trial.params)
    print(f"Trial number: {trial.number}")
    print(f"Parameters: {trial.params}")
    print(f"Value: {trial.value}")
    print(f"---")

## Feature importance catboost

In [ ]:
feature_importance = catboost.get_feature_importance()
feature_names = X_train.columns
names_more_treshold = []
top_feats = []
cnt = 0
# # Display feature importance
for importance, name in sorted(list(zip(feature_importance, feature_names))):
    if importance >= 0.05:
        cnt += 1
        print(f"Feature: {name}, Importance: {importance:.2f}")
        names_more_treshold.append(name)
print(cnt)

## Batch training

In [ ]:
model = cat_classif

batch_size = 512

for i in range(0, len(X_train), batch_size):
    X_batch = X_train[i:i + batch_size]
    y_batch = y_train[i:i + batch_size]
    
    model.fit(X_batch, y_batch, verbose=False)

## Stratified model training

In [ ]:
n_splits = 10
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)
models = []

for train_index, test_index in skf.split(X_train, y_train):
    X_train, X_test = X_train.iloc[train_index], X_train.iloc[test_index]
    y_train, y_test = y_train.iloc[train_index], y_train.iloc[test_index]

    model = catboost.CatBoostClassifier()
    model.fit(X_train, y_train)
    models.append(model)
    
    model.save_model(f'catboost_model_{len(models)}.cbm')

## Read chunks

In [ ]:
destribiution = pd.DataFrame('your_file.csv',  usecols=['Target'])
n_splits = 10 # столько чтобы не полетела оперативка
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
id = [[x] for x in range(len(destribiution))]

for _, chunk_index in skf.split(id, destribiution):
    df_chunk = pd.read_csv('your_file.csv', skiprows=lambda x: x not in id)

# **Xgboost**

## Defult xgboost

In [ ]:
# classification
xgb_classif = xgboost.XGBClassifier(random_state=42, tree_method = 'gpu_hist', device='CUDA')
xgb_classif.fit(X_train, y_train)

In [ ]:
# regression
xgb_reg = xgboost.XGBRegressor(random_state=42, tree_method = 'gpu_hist', device='CUDA')
xgb_reg.fit(X_train, y_train)

## Optuna xgboost

In [ ]:
def objective(trial):
    param = {
        'n_estimators': trial.suggest_int('n_estimators', 500, 5000),
        "objective": trial.suggest_categorical("objective", ["binary:logistic"]),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.01, 1),
        "learning_rate": trial.suggest_float("learning_rate", 1e-3, 1, log=True),
        "max_depth": trial.suggest_int("max_depth", 1, 12),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 100),
        'reg_alpha': trial.suggest_float('reg_alpha', 0.0, 100.0),
        'reg_lambda': trial.suggest_float('reg_lambda', 0.0, 100.0),
        "subsample": trial.suggest_float("subsample", 0.1, 1),
        "gamma": trial.suggest_float("gamma", 0, 10),
        "scale_pos_weight": trial.suggest_float("scale_pos_weight", 1, 10),
        'max_delta_step': trial.suggest_float('max_delta_step', 0.0, 10.0),
        'grow_policy': trial.suggest_categorical('grow_policy', ['depthwise', 'lossguide']),
        'device': 'cuda',  
        'tree_method': 'hist'
    }

    xgb_cls = xgb.XGBClassifier(**param, eval_metric='f1')

    xgb_cls.fit(X_train, y_train, eval_set=[(X_val, y_val)], verbose=0, early_stopping_rounds=500)
    
    preds = xgb_cls.predict(X_test)
    f1 = f1_score(y_test, preds)
    return f1

if __name__ == "__main__":
    study = optuna.create_study(direction="maximize")
    study.optimize(objective, timeout=7200)

    print("Number of finished trials: {}".format(len(study.trials)))

    print("Best trial:")
    trial = study.best_trial

    print("  Value: {}".format(trial.value))

    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))

# **Stacking**

> Код грязный, но пока так

In [ ]:
class DjStacking(BaseEstimator, ClassifierMixin):
    """Стэкинг моделей scikit-learn"""

    def __init__(self, models, ens_model):
        """
        Инициализация
        models - базовые модели для стекинга
        ens_model - мета-модель
        """
        self.models = models
        self.ens_model = ens_model
        self.n = len(models)
        self.valid = None
        
    def fit(self, X, y=None, p=0.25, cv=3, err=0.001, random_state=None, dop_models=None):
        """
        Обучение стекинга
        p - в каком отношении делить на обучение / тест
            если p = 0 - используем всё обучение!
        cv  (при p=0) - сколько фолдов использовать
        err (при p=0) - величина случайной добавки к метапризнакам
        random_state - инициализация генератора
            
        """
        if (p > 0): # делим на обучение и тест
            # разбиение на обучение моделей и метамодели
            train, valid, y_train, y_valid = train_test_split(X, y, test_size=p, random_state=random_state)
            
            # заполнение матрицы для обучения метамодели
            self.valid = np.zeros((valid.shape[0], self.n))
            for t, clf in enumerate(self.models):
                clf.fit(train, y_train)
                self.valid[:, t] = clf.predict_proba(valid)[:, 1]
                
            # обучение метамодели
            self.ens_model.fit(self.valid, y_valid)
            print(f'F1: {round(f1_score(self.ens_model.predict(self.valid), y_valid), 3)}')

        else: # используем всё обучение
        
            # для регуляризации - берём случайные добавки
            self.valid = err*np.random.randn(X.shape[0], self.n)
            
            for t, clf in enumerate(self.models):
                # это oob-ответы алгоритмов
                self.valid[:, t] += cross_val_predict(clf, X, y, cv=cv, method='predict_proba')[:, 1]
                # но сам алгоритм надо настроить
                clf.fit(X, y)
            
            # обучение метамодели
            self.ens_model.fit(self.valid, y)
            print(f'F1: {round(f1_score(self.ens_model.predict(self.valid), y), 3)}')
        

        return self
    

    def predict(self, X, y=None):
        """
        Работа стэкинга
        """
        # заполение матрицы для мета-классификатора
        X_meta = np.zeros((X.shape[0], self.n))
        
        for t, clf in enumerate(self.models):
            X_meta[:, t] = clf.predict(X)
        
        a = self.ens_model.predict(X_meta)
        
        return (a)
    
    def predict_proba(self, X, y=None):
        """
        Работа стэкинга
        """
        # заполение матрицы для мета-классификатора
        X_meta = np.zeros((X.shape[0], self.n))
        
        for t, clf in enumerate(self.models):
            X_meta[:, t] = clf.predict_proba(X)[:, 1]

        a = self.ens_model.predict_proba(X_meta)
        
        return (a)
    
    def fit_ens_model(self, X, y=None, cv=3, err=0.001):
        self.valid = err*np.random.randn(X.shape[0], self.n)
        
        for t, clf in enumerate(self.models):
            self.valid[:, t] += cross_val_predict(clf, X, y, cv=cv, n_jobs=-1, method='predict_proba')[:, 1]

        # Полиномиальные признаки до второй степени

        self.ens_model.fit(self.valid, y)  
        

        return self

In [ ]:
# базовые модели для стекинга

'''
gbm1 = lgb.LGBMClassifier(random_state=54, device="gpu", learning_rate=0.143)

gbm2 = lgb.LGBMClassifier(random_state=8743, device="gpu", learning_rate=0.1)    

gbm3 = lgb.LGBMClassifier(random_state=2367, device="gpu", learning_rate=0.3)

xgb1 = XGBClassifier(random_state=13, tree_method = 'gpu_hist', device='CUDA', learning_rate=0.15)

xgb2 = XGBClassifier(random_state=74, tree_method = 'gpu_hist', device='CUDA', learning_rate=0.1)

xgb3 = XGBClassifier(random_state=788, tree_method = 'gpu_hist', device='CUDA', learning_rate=0.19)
'''
cat1 = catboost.CatBoostClassifier(random_seed=42, verbose=200, eval_metric='F1', task_type="GPU")

cat2 = catboost.CatBoostClassifier(random_seed=472, verbose=200, eval_metric='F1', task_type="GPU")

cat3 = catboost.CatBoostClassifier(random_seed=12, verbose=200, eval_metric='F1', task_type="GPU")

cat4 = catboost.CatBoostClassifier(random_seed=125, verbose=200, eval_metric='F1', task_type="GPU")

cat5 = catboost.CatBoostClassifier(random_seed=132, verbose=200, eval_metric='F1', task_type="GPU")

In [ ]:
#models = [gbm1, gbm2, gbm3, xgb1, xgb2, xgb3, cat1, cat2, cat3]
models = [cat1, cat2, cat3, cat4, cat5]
ens_model = catboost.CatBoostClassifier(verbose=200, task_type="GPU", random_seed=62)

s2 = DjStacking(models, ens_model)
s2.fit(X_train, y_train, p=-1, cv=5, random_state=42)
#print(f'F1: {round(f1_score(y_test, preds), 3)}')

# **Cross_val emsemble**

In [ ]:
# top_trials_params - лучшее из optuna
top_trials_params = []

In [ ]:
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=1)
data = X_train
metrics_stratified = []
cv_models = []
indx = 0
for train_index, test_index in skf.split(data, y_train):
    x_train_fold, x_test_fold = data.iloc[train_index, :], data.iloc[test_index, :]
    y_train_fold, y_test_fold = y_train[train_index], y_train[test_index]
    cat_cv = catboost.CatBoostClassifier(**top_trials_params[indx], eval_metric='F1')
    cat_cv.fit(x_train_fold, y_train_fold,
             verbose=100, early_stopping_rounds=500)
    pred = cat_cv.predict(x_test_fold)
    metrics_stratified.append((f1_score(pred, y_test_fold).round(3), roc_auc_score(pred, y_test_fold).round(3)))
    cv_models.append(cat_cv)
    indx += 1

print('\n'.join(map(str, metrics_stratified)))

In [ ]:
# Для большего  кол-ва моделей, для которых фолдов уже не хватает
# Здесь несколько моделей обучаются на одинаковых фолдах
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=1)
data = X_train
metrics_stratified = []
cv_models = []
indx = 0
for train_index, test_index in skf.split(data, y_train):
    x_train_fold, x_test_fold = data.iloc[train_index, :], data.iloc[test_index, :]
    y_train_fold, y_test_fold = y_train[train_index], y_train[test_index]

    catboost1 = catboost.CatBoostClassifier(**top_trials_params[indx], eval_metric='F1')
    catboost2 = catboost.CatBoostClassifier(**top_trials_params[indx + 1], eval_metric='F1')
    catboost3 = catboost.CatBoostClassifier(**top_trials_params[indx + 2], eval_metric='F1')

    catboost1.fit(x_train_fold, y_train_fold, verbose=300, early_stopping_rounds=500),
    catboost2.fit(x_train_fold, y_train_fold, verbose=300, early_stopping_rounds=500)
    catboost3.fit(x_train_fold, y_train_fold, verbose=300, early_stopping_rounds=500)

    pred1, pred2, pred3 = catboost1.predict(x_test_fold), catboost2.predict(x_test_fold), catboost3.predict(x_test_fold)
    metrics_stratified.append((f1_score(pred1, y_test_fold).round(3), roc_auc_score(pred1, y_test_fold).round(3)))
    metrics_stratified.append((f1_score(pred2, y_test_fold).round(3), roc_auc_score(pred2, y_test_fold).round(3)))
    metrics_stratified.append((f1_score(pred3, y_test_fold).round(3), roc_auc_score(pred3, y_test_fold).round(3)))
    cv_models.append(catboost1)
    cv_models.append(catboost2)
    cv_models.append(catboost3)
    indx += 3

print('\n'.join(map(str, metrics_stratified)))

In [ ]:
mean_auc = 0
mean_f1 = 0
for metric in metrics_stratified:
    mean_auc += metric[1]
    mean_f1 += metric[0]

print('ROC_AUC:', (mean_auc / len(metrics_stratified)).round(3))
print('F1:', (mean_f1 / len(metrics_stratified)).round(3))

In [ ]:
# Cross Val predict
preds = []
for model in cv_models:
    pred = model.predict_proba(X_test)[:,1]
    preds.append(pred)

arr_np = np.array(preds)
mean_arr = np.mean(arr_np, axis=0)

pred = (mean_arr >= 0.5).astype(int)

# **Data preprocess**

## Fill NaN

In [ ]:
def fillna_mice(trainX):
    scaler = StandardScaler()
    trainX_mice = trainX.copy()
    trainX_mice = pd.DataFrame(scaler.fit_transform(trainX_mice), columns = trainX.columns)
    mice_imputer = IterativeImputer(initial_strategy = 'mean',
                                    estimator = LinearRegression(n_jobs=-1),
                                    random_state = 42, verbose=2, max_iter=10)

    mice = mice_imputer.fit_transform(trainX_mice)
    return pd.DataFrame(scaler.inverse_transform(mice), columns = trainX.columns), mice_imputer, scaler


# new dataframe
trainX_mice, mice_imputer, scaler = fillna_mice(X_train)
trainX_mice.head()

In [ ]:
# Simple Imputer
imputer = SimpleImputer(strategy='mean')
X_simple = imputer.fit_transform(X_train)
X_simple = pd.DataFrame(X_simple, columns = X_train.columns)

## Feature engineering

In [ ]:
poly_features = X_train[top_feats] # Предварительно отобрать топ
poly_transformer = PolynomialFeatures(degree = 3)

poly_transformer.fit(poly_features)
poly_features = poly_transformer.transform(poly_features)
print('Polynomial Features shape: ', poly_features.shape)

## Feature selection

In [ ]:
# статичные фичи
sel = VarianceThreshold(threshold=0)
columns_df = X_train.columns
sel.fit(X_train)
get_sup_col = sel.get_support()

# мультикор.
df_transformed = sel.transform(X_train)
df_transformed = pd.DataFrame(df_transformed, columns=columns_df[get_sup_col])
corr_matrix = df_transformed.corr()
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
to_drop = [column for column in upper.columns if any(upper[column] > 0.9)]
df_transformed = df_transformed.drop(to_drop, axis=1)

## Class Disbalance

> Если метрика F1, то лучше будет просто использовать class_weights + подбор трешхолда

In [ ]:
# Создание экземпляра RandomOverSampler
ros = RandomOverSampler()

# Применение увеличения выборки к данным
X_resampled, y_resampled = ros.fit_resample(X_train, y_train)

In [ ]:
# Создание экземпляра RandomUnderSampler
rus = RandomUnderSampler()

# Применение уменьшения выборки к данным
X_resampled, y_resampled = rus.fit_resample(X_train, y_train)

In [ ]:
# Создание экземпляра SMOTE
smote = SMOTE()

# Применение SMOTE к данным
X_resampled, y_resampled = smote.fit_resample(X_train, y_train)

# **Other**

## Treshold

In [ ]:
pred = cat_reg.predict_proba(X_val)[:, 1]
precision, recall, thresholds = precision_recall_curve(y_val, pred)
fscore = (2 * precision * recall) / (precision + recall)
best_threshold = thresholds[np.argmax(fscore)]
print(f"Best threshold: {best_threshold}")

## Validation model

In [ ]:
model = catboost
pred = model.predict(X_val)
# pred = [1 if x >= 0.5 else 0 for x in pred]
print(classification_report(pred, y_val))
print(f'F1_score: {f1_score(pred, y_val).round(3)}')
print(f'Roc_auc: {roc_auc_score(pred, y_val).round(3)}')

## Stratified cross-val

In [ ]:
model = cat_classif
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

cv_scores = cross_val_score(model, X_train, y_train, cv=skf, scoring='f1_macro')

# Выводим результаты
print("Стратифицированная кросс-валидация F1-метрика (macro):", cv_scores)
print("Среднее значение F1:", np.mean(cv_scores))
print("Стандартное отклонение F1:", np.std(cv_scores))